In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import spacy
import pytextrank  # We're not going to execute this at home.
import pprint
from collections import Counter
from statistics import mean, median

In [2]:
#TF-IDF Function
def Tfidf(seg_list:list, top_k:int):
    vectorizer = CountVectorizer() # 建立 CountVectorizer 物件
    word_count = vectorizer.fit_transform(seg_list) # 將 text 轉成詞頻矩陣
    tfidf_transformer = TfidfTransformer() # 建立 TfidfTransformer 物件
    tfidf_matrix = tfidf_transformer.fit_transform(word_count) # 將詞頻矩陣轉換成 TF-IDF 矩陣
    words = vectorizer.get_feature_names() # 取得詞彙表
    tfidf_values = tfidf_matrix.toarray()[0] # 取得每個詞彙的 TF-IDF 值
    sorted_index = np.argsort(tfidf_values)[::-1] # 取得排序後的索引位置
    tfidf_keywords = [words[i] for i in sorted_index[:top_k]] # 取得排名前 K位 的關鍵字
    return tfidf_keywords

#TextRank Function
def Txtrank(seg_list:list, top_k:int):
    nlp = spacy.load('zh_core_web_sm')
    nlp.add_pipe("textrank")
    doc = nlp(seg_list)
    textrank_keywords = []
    for p in doc._.phrases:
        if len(textrank_keywords) >= top_k:
            break
        textrank_keywords.append(p.text)
    return(textrank_keywords)

#LDA Function

def Lda(seg_list:list, Num_topics:int, Num_keywords:int, Max_iter:int):
    vectorizer = CountVectorizer() # 建立CountVectorizer物件
    X = vectorizer.fit_transform(seg_list) # 使用CountVectorizer物件將文本轉換成詞頻矩陣
    lda = LatentDirichletAllocation(n_components=Num_topics, max_iter=Max_iter)
    lda.fit_transform(X)
    for topic_idx, topic in enumerate(lda.components_): # 取出每個主題的關鍵字
        top_keyword_idxs = topic.argsort()[:-Num_keywords-1:-1]
        top_keywords = [vectorizer.get_feature_names()[idx] for idx in top_keyword_idxs]
    return top_keywords

def topic_list(segs:list, idxs:list): #idxs是主題內每篇文章的index所組成的list
    topic = []
    for n in idxs:
        topic.append(segs[n-1]) #List Index是從0開始，所以DF Index要-1
    return topic
    
def count_seg(topic_segs:list):
    k = Counter(topic_segs).keys()
    v = Counter(topic_segs).values()
    res = dict(zip(k, v))
    sort_res = dict(sorted(res.items(), key=lambda x: x[1], reverse=True))
    return(sort_res)

#用LDA批次抓詞的func，輸入要拿來抓詞的母體，還有K設定抓多少個詞
def lda_keys(seg_list, kv):
    lda_list = []
    for sg in seg_list:
        kw = Lda(sg, 1, kv, 50)
        lda_list.append(kw)
    return lda_list

#計算每個語料的分詞有多少個
def count_elements(seg_list):
    element_count = []
    for em in seg_list:
        count = len(em)
        element_count.append(count)
    return element_count

In [3]:
data_seg = pd.read_csv('./Original_data/taiwan_charity_news_seglist.csv', encoding='UTF-8', index_col='Index')

In [4]:
data_seg.tail(3)

,Date,Title,Media,Content,Related,Topic_test,News_tags,Stance_label,Seg_list,Title_seg
Index,,,,,,,,,,
443,2020/10/5,赤字問題如何解？庇護工場的經營與難處。,Peoponews,庇護工場的廚房內傳出陣陣餅乾香氣，每位唐氏症患者都專注地工作著。這一間間的庇護工場，是政府核...,0,0,0,unrelated,庇護工場 廚房 內 傳出 陣陣 餅乾 香氣 每 位 唐氏症 患者 專注 工作 一 間間 庇護...,赤字 問題 解 庇護工場 經營 難處
444,2021/6/3,疫情慘烈，唐氏症基金會5月募款短少35%、庇護工場損失逾百萬。,Chinatimes,中央流行疫情指揮中心頒布三級警戒，各縣市餐廳禁止內用，全面衝擊各個產業。據唐氏症基金會統計，...,0,0,0,unrelated,中央 流行 疫情 指揮 中心 頒布 三 級 警戒 各 縣市 餐廳 禁止 內用 衝擊 各 個 ...,疫情 慘烈 唐氏症 基金會 5月 募款 短少 35% 庇護工場 損失 逾 百萬
445,2022/10/4,公益大使柯佳嬿籲公益捐款，身心障礙服務善款缺口達4千萬。,Money_UDN,台灣首個身心障礙者家長自組團體「心路基金會」於今年成立達 35 週年，600人團隊每年服務6...,0,0,0,unrelated,台灣 首 個 身心 障礙 者 家長 自組 團體 心路 基金會 今年 成立 達 35 週年 ...,公益 大使 柯佳嬿 籲 公益 捐款 身心 障礙 服務 善款 缺口 達 4千萬


In [5]:
data_seg['Seg_list'][445]

'台灣 首 個 身心 障礙 者 家長 自組 團體 心路 基金會 今年 成立 達  35 週年 600 人 團隊 每 年 服務 6千 位 心智 障礙 者 35 年 來 累積 協助 高達 3.7萬 個 家庭 讓 照顧 者 安心 重回 工作 崗位 去年 2021年 為 例 心路 服務 幫助 5300 個 身心 障礙 者 家庭 讓 照顧 者 重返 職場 等於 社會 貢獻 2.47億 經濟 效益 心路 輔導 就業 工作 產值 佔 1.27億 疫情 不景氣 期間 心路 面臨 捐款 缺口 達 4千萬 元 各 項 計畫 進行 停歇 今日 特別 邀請 金鐘 影后 柯佳嬿 擔任 心路 公益 大使 呼籲 各界 熱心 捐款 支持 35 週年 心路 提出 好 核心 信念 心路 基金會 執行長 賴炳良 表示 好 理念 疫情 逆境 中 全 台灣 共享  35 年 來 心路 完成 許多 創舉 身心 障礙 者 權利 公約 說 聯合國 2008年 生效 後 心路 2011年 推出 社區 生活 支持 服務 台灣 2014年 立法 早 三 年 讓 身心 障礙 者 學習 自立 社區 接軌 生活 有 品質 有 尊嚴 疫情 期間 機構 參訪 暫停 心路 輔導 心智 障礙 者 自立 工作 場域 庇護工場 產線 線上化 讓 各 地 機構 團體 支持 者 們 線上 參訪 學習 心路 經驗 同時 過去  年 支持 障礙 者 累積 發明 近 百 套 教材 輔具 精選 33 件 分享 心路網站 上 歡迎 家長 從業 人員 參考 運用 有 一 項 輔具 平時 心路 成人 機構 用於 讓 障礙 者 投票 決定 事務 公益 大使 柯佳嬿 活動 上 示範 使用 投票 輔具 賴炳良 執行長 心路 服務 對象 票選 晚餐 象徵 鼓勵 身心 障礙 者 表達 意見 決定 生活  5 歲 個案 星橙 為 例 腦麻 影響 學習 照顧 者 花上 好幾 倍 時間 陪伴 心路 基金會 專業 輔導 後 讓 星橙 學習 抓取 食物 放入 嘴巴 看似 平凡 動作 星橙 一家人 教育 里程碑 心路 協助 讓 星橙媽 有 心力 攻讀 特殊 教育 碩士 完成 碩士 人生 目標 未來 期待 投入 社會 工作 服務 更多 特教 孩童 讓 障礙 者 家庭 社會 三 方 形成 循環 心路 接受 超過 20 年 服務 唐寶寶子娟 現身 分享 子娟 參與 合唱團 話劇課 

In [117]:
hostile = data_seg.loc[data_seg['Stance_label']=='hostile']

In [9]:
enemy = []
for seg in hostile['Seg_list']:
    sg = seg.split(' ')
    enemy.append(sg)

In [6]:
# 把斷詞好的文章重新從Dataframe裡面拿出來，由於存進去的時候被強制轉換成整串str，所以這邊重新轉換回List，並存到另一個List裡面。
segs = []

for seg in data_seg['Seg_list']:
    sg = seg.split(' ')
    segs.append(sg)

In [7]:
#確認斷詞文章，要注意List的index是從0開始，但是DF的index是從1開始，所以這邊List index都請-1
x = segs[244]
if isinstance(x, list):
    print("yes")
else:
    print("no")
print(x)

yes
['景氣', '低迷', '各行各業', '面臨', '緊縮', '際', '服務', '視障者', '三十多', '年', '愛盲', '基金會', '募款', '遭逢', '重挫', '今年', '四月', '起', '每', '月', '募款', '往年', '降', '一', '二成', '捐款', '縮水', '全', '台', '有', '近', '五萬', '名', '中途', '失明', '者', '急需', '重建', '生活', '愛盲', '今日', '發起', '視障', '生活', '重建', '服務', '計劃', '中途', '失明', '者', '家庭', '提供', '全人', '全', '程', '全', '隊', '五', '大', '服務', '邀請', '代言人', '李國毅', '呼籲', '希望', '愛心', '企業', '磚引', '玉', '關懷', '視障', '朋友', '衛福部', '統計', '全', '台', '五萬五千多', '名', '視障者', '中', '中途', '失明', '者', '佔', '八成七', '五萬', '人', '反映出', '五萬', '個', '家庭', '中途', '失明', '者', '承受', '視障', '帶來', '生活', '劇變', '愛盲', '基金會', '執行長', '劉怡君', '表示', '中途', '失明', '一', '個', '人', '事', '一家人', '事', '許多', '中途', '失明', '者', '視力', '喪失', '初期', '需要', '親友', '大量', '陪伴', '愛盲', '完整', '專業', '團隊', '服務', '協助', '家屬', '理解', '視障者', '視力', '退化', '心理', '生活', '行走', '狀態', '建立', '溝通', '橋樑', '讓', '視障者', '家屬', '低', '學習', '壓力', '情況', '下', '樂於', '參與', '重建', '學習', '愛盲', '今年', '邀請到', '李國毅', '擔任', '代言人', '入鏡', '練習題', '公益', '廣告', '廣告', '內容', '描述', '一', '位', '適應'

In [51]:
#計算Content中每個語料的分詞有多少個
element_count = count_elements(segs)

In [79]:
print("All the seg_lists are", element_count) 
print("We have", len(element_count), "seg_lists")
print("Max length of Seg_list is", max(element_count))
print("Min length of Seg_list is", min(element_count))
print("Mean length of Seg_list is", mean(element_count))
print("Median length of Seg_list is", median(element_count))
print("Quantile of Seg_list are", np.quantile(element_count, q = np.arange(0.25, 1, 0.25)))

All the seg_lists are [234, 258, 251, 170, 146, 502, 419, 433, 197, 161, 223, 197, 88, 210, 263, 238, 265, 402, 273, 125, 130, 154, 65, 173, 345, 124, 176, 114, 196, 80, 113, 218, 329, 247, 524, 135, 170, 140, 188, 236, 624, 227, 396, 141, 128, 751, 832, 271, 236, 125, 158, 141, 277, 125, 209, 136, 233, 201, 188, 99, 193, 384, 191, 689, 408, 185, 253, 212, 202, 241, 217, 233, 104, 502, 309, 249, 177, 610, 230, 187, 155, 152, 151, 308, 176, 283, 214, 120, 177, 442, 460, 592, 162, 488, 162, 270, 301, 270, 109, 148, 618, 194, 284, 348, 209, 419, 189, 125, 257, 400, 198, 295, 521, 242, 475, 280, 365, 170, 153, 181, 199, 340, 258, 227, 262, 299, 136, 214, 215, 180, 136, 217, 229, 229, 172, 131, 206, 264, 182, 206, 170, 188, 468, 158, 256, 298, 204, 302, 803, 225, 135, 588, 190, 216, 224, 71, 353, 223, 124, 304, 229, 88, 210, 418, 230, 230, 494, 323, 220, 692, 450, 246, 836, 172, 307, 215, 360, 329, 158, 534, 1367, 294, 124, 453, 537, 185, 209, 163, 286, 487, 203, 345, 62, 222, 210, 348, 219

In [110]:
def check_emsize(element_count, standard):
    smaller_count = 0
    bigger_count = 0
    for element in element_count:
        if element < standard:
            smaller_count += 1
        elif element > standard:
            bigger_count += 1
    return smaller_count, bigger_count

In [114]:
check_emsize(element_count, 30)

(1, 444)

In [31]:
# Topic1_idx = [96,97,98,101,107]
# Topic2_idx = [128,129,130,131,132,133,134,135]
# Topic3_idx = [244,245,246,247,248,249,250,251,252,253,256, 270]
# Topic4_idx = [260,262,263,264,265]

In [32]:
# Topic_1 = topic_list(segs, Topic1_idx)
# Topic_2 = topic_list(segs, Topic2_idx)
# Topic_3 = topic_list(segs, Topic3_idx)
# Topic_4 = topic_list(segs, Topic4_idx)

In [ ]:
#幫助計算四個Topic中每則內容的詞頻
# count_seg(Topic_4[4])

In [ ]:
# tfidf_list = []
# for sg in segs:
#     x= Tfidf(sg, k_value)
#     tfidf_list.append(x)

In [ ]:
# print(tfidf_list[0])

In [ ]:
# txtrank_list =[]
# for sg in data_seg['Seg_list']:
#     y= Txtrank(sg, k_value)
#     txtrank_list.append(y)

In [ ]:
# print(txtrank_list[97])

In [97]:
lda_list = []
for sg in enemy:
    z= Lda(sg, 1, 30, 50)
    lda_list.append(z)

In [ ]:
print(lda_list[0])

In [10]:
LDA_Key_K6 = lda_keys(segs, 6)

In [13]:
print(LDA_Key_K6[2])

['患者', '白化症', '黑色素', '視力', '資源', '視覺']


In [14]:
LDA_Key_K7 = lda_keys(segs, 7)

In [16]:
LDA_Key_K8 = lda_keys(segs, 8)

In [17]:
LDA_Key_K9 = lda_keys(segs, 9)

In [18]:
LDA_Key_K10 = lda_keys(segs, 10)

In [19]:
LDA_Key_K11 = lda_keys(segs, 11)

In [20]:
LDA_Key_K12 = lda_keys(segs, 12)

In [8]:
LDA_Key_K18 = lda_keys(segs, 18)

In [ ]:
LDA_Key_K20 = lda_keys(segs, 20)

In [ ]:
LDA_Key_K25 = lda_keys(segs, 25)

In [ ]:
LDA_Key_K30 = lda_keys(segs, 30)

In [ ]:
LDA_Key_K35 = lda_keys(segs, 35)

In [21]:
data_seg['Seg_counts'] = element_count
data_seg['Keyword_K6'] = LDA_Key_K6
data_seg['Keyword_K7'] = LDA_Key_K7
data_seg['Keyword_K8'] = LDA_Key_K8
data_seg['Keyword_K9'] = LDA_Key_K9
data_seg['Keyword_K10'] = LDA_Key_K10
data_seg['Keyword_K11'] = LDA_Key_K11
data_seg['Keyword_K12'] = LDA_Key_K12
data_seg['Keyword_K18'] = LDA_Key_K18
data_seg['Keyword_K20'] = LDA_Key_K20
data_seg['Keyword_K25'] = LDA_Key_K25
data_seg['Keyword_K30'] = LDA_Key_K30
data_seg['Keyword_K35'] = LDA_Key_K35

In [22]:
data_seg.head(3)

,Date,Title,Media,Content,Related,Topic_test,Stance_label,Seg_list,News_tags,Keyword_K6,Keyword_K7,Keyword_K8,Keyword_K9,Keyword_K10,Keyword_K11,Keyword_K12
Index,,,,,,,,,,,,,,,,
1,2017/1/9,視障弟月薪僅800？勞工局：是見習生獎金，不是薪資。,ETtoday_News,一名雙眼失明的闕小弟因為父親入獄、肢體障礙的母親也遠在高雄，只剩他和弟弟2人在北部生活，只好...,1,0,neutral,一 名 雙眼 失明 闕 小弟 父親 入獄 肢體 障礙 母親 遠 高雄 剩 弟弟 2 人 北部...,0,"[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時]","[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時, 生活]","[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時, 生活, 擔任]","[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時, 生活, 擔任, 協助]","[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時, 生活, 擔任, 協助, 目前]","[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時, 生活, 擔任, 協助, 目前, 就業]","[庇護工場, 勞工局, 小弟, 薪資, 愛盲, 當時, 生活, 擔任, 協助, 目前, 就業..."
2,2017/1/13,開發金，邀九企業挺公益。,UDN_Database,開發金控多年來透過旗下中華開發工銀文教基金會及凱基社會慈善基金會，辦理各項公益活動。「年度公...,1,0,neutral,開發 金控 多 年 來 旗下 中華 開發 工銀 文教 基金會 凱基 社會 基金會 辦理 各 ...,0,"[公益, 社會, 開發, 基金會, 企業, 金控]","[公益, 社會, 開發, 基金會, 企業, 金控, 參與]","[公益, 社會, 開發, 基金會, 企業, 金控, 工銀, 參與]","[公益, 社會, 開發, 基金會, 企業, 金控, 參與, 工銀, 文教]","[公益, 社會, 開發, 基金會, 企業, 金控, 參與, 工銀, 文教, 鼓勵]","[公益, 社會, 開發, 基金會, 企業, 金控, 參與, 工銀, 文教, 鼓勵, 具體]","[公益, 社會, 開發, 基金會, 企業, 金控, 參與, 工銀, 文教, 鼓勵, 具體, 員工]"
3,2017/3/3,月亮的孩子加強視力，把握黃金期。,UDN_Database,白化症患者體內缺乏黑色素，頭髮和皮膚雪白，因眼睛畏光、在夜間活動相對較舒適，印地安人稱他們為...,1,0,neutral,白化症 患者 體 內 缺乏 黑色素 頭髮 皮膚 雪白 眼睛 畏 光 夜間 活動 相對 舒適 ...,0,"[患者, 白化症, 黑色素, 視力, 資源, 視覺]","[患者, 白化症, 黑色素, 視力, 資源, 視覺, 小弟]","[患者, 白化症, 黑色素, 視力, 資源, 視覺, 小弟, 林育葳]","[患者, 白化症, 黑色素, 視力, 資源, 視覺, 小弟, 林育葳, 孩子]","[患者, 白化症, 黑色素, 視力, 資源, 視覺, 小弟, 林育葳, 孩子, 發育]","[患者, 白化症, 黑色素, 視力, 資源, 視覺, 小弟, 林育葳, 孩子, 發育, 其他]","[患者, 白化症, 黑色素, 視力, 資源, 視覺, 小弟, 林育葳, 孩子, 發育, 其他..."
